**BÀI TẬP LỚN  MÔN HỌC MÁY**

19021374 - Phạm Thị Minh Trang


# Mô tả bài toán
Phân loại các câu hỏi thật (sincere) và câu hỏi chỉ mang tính câu fame (insincere).

**Input**: Các câu hỏi ở dạng text

**Output**: 0 nếu câu hỏi là sincere và 1 nếu là insincere.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Phân tích dữ liệu
Dữ liệu được cung cấp sẵn gồm có hai file csv để lưu dữ liệu train và test, một file zip chứa 4 loại pretrained word embeddings.

In [ ]:
train_path = '../input/quora-insincere-questions-classification/train.csv'
test_path = '../input/quora-insincere-questions-classification/test.csv'
emb_path = '../input/quora-insincere-questions-classification/embeddings.zip'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

Dữ liệu huấn luyện gồm có ba trường: 
- qid: đánh index định danh cho câu hỏi. Không ảnh hưởng gì đến việc huấn luyện.
- question_text: nội dung câu hỏi cần phải phân loại.
- target: phân loại của câu hỏi trên. Giá trị 0 là sincere, 1 là insincere.

In [ ]:
train_data.head()

Tập dữ liệu huấn luyện có 1306122 mẫu, được phân vào 2 lớp (0 và 1). Trong đó, có 1225312 thuộc lớp 0, 80810 thuộc lớp 1.

Trung bình một mẫu có độ dài 11 từ

In [ ]:
# số lượng các mẫu trong tập huấn luyện
num_sample = len(train_data)
print('Number of training samples: ', num_sample)
print('Number of classes', max(train_data['target'])+1)

num_word = [len(s.split()) for s in train_data['question_text']]
# trung bình số từ trong một mẫu
num_wps = np.median(num_word)
print('Number of words per sample: ', num_wps)


In [ ]:
train_data['target'].value_counts()

In [ ]:
# Một cái biểu đồ đẹp đẽ thể hiện sự mất cân bằng giữa hai phân lớp trong tập dữ liệu train
import matplotlib.pyplot as plt

plt.figure(figsize=(10,7))
plt.subplot(1, 2, 1)
plt.title('Question count')
bar = plt.bar(['Sincere', 'Insincere'], [train_data.loc[train_data['target'] == 0].count()[0], train_data.loc[train_data['target'] == 1].count()[0]])

In [ ]:
# # cắt bớt data để 2 nhóm cân bằng
# from random import sample
# train_data_balance = train_data[train_data['target'] == 1]
# train_data_balance = train_data_balance.append(train_data[train_data['target'] == 0].sample(len(train_data[train_data['target']==1])))\n
# # shuffle all rows and reset index columns
# train_data_balance = train_data_balance.sample(frac=1).reset_index(drop=True)
# train_data_balance

# Làm sạch dữ liệu
- Xóa ngắt câu và kí tự lạ
- Xóa số
- Sửa sai chính tả
- Viết thường

In [ ]:
punctuation_list = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', 
        '•', '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 
        '█', '…', '“', '★', '”', '–', '●', '►', '−', '¢', '¬', '░', '¡', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
        '—', '‹', '─', '▒', '：', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '¸', '⋅', '‘', '∞', 
        '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '・', '╦', '╣', '╔', '╗', '▬', '❤', '≤', '‡', '√', '◄', '━', 
        '⇒', '▶', '≥', '╝', '♡', '◊', '。', '✈', '≡', '☺', '✔', '↵', '≈', '✓', '♣', '☎', '℃', '◦', '└', '‟', '～', '！', '○', 
        '◆', '№', '♠', '▌', '✿', '▸', '⁄', '□', '❖', '✦', '．', '÷', '｜', '┃', '／', '￥', '╠', '↩', '✭', '▐', '☼', '☻', '┐', 
        '├', '«', '∼', '┌', '℉', '☮', '฿', '≦', '♬', '✧', '〉', '－', '⌂', '✖', '･', '◕', '※', '‖', '◀', '‰', '\x97', '↺', 
        '∆', '┘', '┬', '╬', '،', '⌘', '⊂', '＞', '〈', '⎙', '？', '☠', '⇐', '▫', '∗', '∈', '≠', '♀', '♔', '˚', '℗', '┗', '＊', 
        '┼', '❀', '＆', '∩', '♂', '‿', '∑', '‣', '➜', '┛', '⇓', '☯', '⊖', '☀', '┳', '；', '∇', '⇑', '✰', '◇', '♯', '☞', '´', 
        '↔', '┏', '｡', '◘', '∂', '✌', '♭', '┣', '┴', '┓', '✨', '\xa0', '˜', '❥', '┫', '℠', '✒', '［', '∫', '\x93', '≧', '］', 
        '\x94', '∀', '♛', '\x96', '∨', '◎', '↻', '⇩', '＜', '≫', '✩', '✪', '♕', '؟', '₤', '☛', '╮', '␊', '＋', '┈', '％', 
        '╋', '▽', '⇨', '┻', '⊗', '￡', '।', '▂', '✯', '▇', '＿', '➤', '✞', '＝', '▷', '△', '◙', '▅', '✝', '∧', '␉', '☭', 
        '┊', '╯', '☾', '➔', '∴', '\x92', '▃', '↳', '＾', '׳', '➢', '╭', '➡', '＠', '⊙', '☢', '˝', '∏', '„', '∥', '❝', '☐', 
        '▆', '╱', '⋙', '๏', '☁', '⇔', '▔', '\x91', '➚', '◡', '╰', '\x85', '♢', '˙', '۞', '✘', '✮', '☑', '⋆', 'ⓘ', '❒', 
        '☣', '✉', '⌊', '➠', '∣', '❑', '◢', 'ⓒ', '\x80', '〒', '∕', '▮', '⦿', '✫', '✚', '⋯', '♩', '☂', '❞', '‗', '܂', '☜', 
        '‾', '✜', '╲', '∘', '⟩', '＼', '⟨', '·', '✗', '♚', '∅', 'ⓔ', '◣', '͡', '‛', '❦', '◠', '✄', '❄', '∃', '␣', '≪', '｢', 
        '≅', '◯', '☽', '∎', '｣', '❧', '̅', 'ⓐ', '↘', '⚓', '▣', '˘', '∪', '⇢', '✍', '⊥', '＃', '⎯', '↠', '۩', '☰', '◥', 
        '⊆', '✽', '⚡', '↪', '❁', '☹', '◼', '☃', '◤', '❏', 'ⓢ', '⊱', '➝', '̣', '✡', '∠', '｀', '▴', '┤', '∝', '♏', 'ⓐ', 
        '✎', ';', '␤', '＇', '❣', '✂', '✤', 'ⓞ', '☪', '✴', '⌒', '˛', '♒', '＄', '✶', '▻', 'ⓔ', '◌', '◈', '❚', '❂', '￦', 
        '◉', '╜', '̃', '✱', '╖', '❉', 'ⓡ', '↗', 'ⓣ', '♻', '➽', '׀', '✲', '✬', '☉', '▉', '≒', '☥', '⌐', '♨', '✕', 'ⓝ', 
        '⊰', '❘', '＂', '⇧', '̵', '➪', '▁', '▏', '⊃', 'ⓛ', '‚', '♰', '́', '✏', '⏑', '̶', 'ⓢ', '⩾', '￠', '❍', '≃', '⋰', '♋', 
        '､', '̂', '❋', '✳', 'ⓤ', '╤', '▕', '⌣', '✸', '℮', '⁺', '▨', '╨', 'ⓥ', '♈', '❃', '☝', '✻', '⊇', '≻', '♘', '♞', 
        '◂', '✟', '⌠', '✠', '☚', '✥', '❊', 'ⓒ', '⌈', '❅', 'ⓡ', '♧', 'ⓞ', '▭', '❱', 'ⓣ', '∟', '☕', '♺', '∵', '⍝', 'ⓑ', 
        '✵', '✣', '٭', '♆', 'ⓘ', '∶', '⚜', '◞', '்', '✹', '➥', '↕', '̳', '∷', '✋', '➧', '∋', '̿', 'ͧ', '┅', '⥤', '⬆', '⋱', 
        '☄', '↖', '⋮', '۔', '♌', 'ⓛ', '╕', '♓', '❯', '♍', '▋', '✺', '⭐', '✾', '♊', '➣', '▿', 'ⓑ', '♉', '⏠', '◾', '▹', 
        '⩽', '↦', '╥', '⍵', '⌋', '։', '➨', '∮', '⇥', 'ⓗ', 'ⓓ', '⁻', '⎝', '⌥', '⌉', '◔', '◑', '✼', '♎', '♐', '╪', '⊚', 
        '☒', '⇤', 'ⓜ', '⎠', '◐', '⚠', '╞', '◗', '⎕', 'ⓨ', '☟', 'ⓟ', '♟', '❈', '↬', 'ⓓ', '◻', '♮', '❙', '♤', '∉', '؛', 
        '⁂', 'ⓝ', '־', '♑', '╫', '╓', '╳', '⬅', '☔', '☸', '┄', '╧', '׃', '⎢', '❆', '⋄', '⚫', '̏', '☏', '➞', '͂', '␙', 
        'ⓤ', '◟', '̊', '⚐', '✙', '↙', '̾', '℘', '✷', '⍺', '❌', '⊢', '▵', '✅', 'ⓖ', '☨', '▰', '╡', 'ⓜ', '☤', '∽', '╘', 
        '˹', '↨', '♙', '⬇', '♱', '⌡', '⠀', '╛', '❕', '┉', 'ⓟ', '̀', '♖', 'ⓚ', '┆', '⎜', '◜', '⚾', '⤴', '✇', '╟', '⎛', 
        '☩', '➲', '➟', 'ⓥ', 'ⓗ', '⏝', '◃', '╢', '↯', '✆', '˃', '⍴', '❇', '⚽', '╒', '̸', '♜', '☓', '➳', '⇄', '☬', '⚑', 
        '✐', '⌃', '◅', '▢', '❐', '∊', '☈', '॥', '⎮', '▩', 'ு', '⊹', '‵', '␔', '☊', '➸', '̌', '☿', '⇉', '⊳', '╙', 'ⓦ', 
        '⇣', '｛', '̄', '↝', '⎟', '▍', '❗', '״', '΄', '▞', '◁', '⛄', '⇝', '⎪', '♁', '⇠', '☇', '✊', 'ி', '｝', '⭕', '➘', 
        '⁀', '☙', '❛', '❓', '⟲', '⇀', '≲', 'ⓕ', '⎥', '\u06dd', 'ͤ', '₋', '̱', '̎', '♝', '≳', '▙', '➭', '܀', 'ⓖ', '⇛', '▊', 
        '⇗', '̷', '⇱', '℅', 'ⓧ', '⚛', '̐', '̕', '⇌', '␀', '≌', 'ⓦ', '⊤', '̓', '☦', 'ⓕ', '▜', '➙', 'ⓨ', '⌨', '◮', '☷', 
        '◍', 'ⓚ', '≔', '⏩', '⍳', '℞', '┋', '˻', '▚', '≺', 'ْ', '▟', '➻', '̪', '⏪', '̉', '⎞', '┇', '⍟', '⇪', '▎', '⇦', '␝', 
        '⤷', '≖', '⟶', '♗', '̴', '♄', 'ͨ', '̈', '❜', '̡', '▛', '✁', '➩', 'ா', '˂', '↥', '⏎', '⎷', '̲', '➖', '↲', '⩵', '̗', '❢', 
        '≎', '⚔', '⇇', '̑', '⊿', '̖', '☍', '➹', '⥊', '⁁', '✢']

# Chứa cả các từ viết sai chính tả, các biến thể us uk, và các từ viết tắt
misspell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 
                 'counselling': 'counseling','theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 
                 'organisation': 'organization', 'wwii': 'world war 2','citicise': 'criticize', 'youtu ': 'youtube ', 
                 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist',
                 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 
                 'whydo': 'why do','doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 
                 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'bitcoin', 
                 'narcissit': 'narcissist', 'bigdata': 'big data','2k17': '2017', '2k18': '2018', 'qouta': 'quota', 
                 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what','watsapp': 'whatsapp', 
                 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization',                 'electroneum':'bitcoin','nanodegree':'degree','hotstar':'star','dream11':'dream','ftre':'fire','tensorflow':'framework',                'unocoin':'bitcoin','lnmiit':'limit','unacademy':'academy','altcoin':'bitcoin','altcoins':'bitcoin','litecoin':'bitcoin',                'coinbase':'bitcoin','cryptocurency':'cryptocurrency','simpliv':'simple','quoras':'quora','schizoids':'psychopath',                'remainers':'remainder','twinflame':'soulmate','quorans':'quora','brexit':'demonetized','iiest':'institute','dceu':'comics',                'pessat':'exam','uceed':'college','bhakts':'devotee','boruto':'anime','cryptocoin':'bitcoin','blockchains':'blockchain',                'fiancee':'fiance','redmi':'smartphone','oneplus':'smartphone','qoura':'quora','deepmind':'framework','ryzen':'cpu',                'whattsapp':'whatsapp','undertale':'adventure','zenfone':'smartphone','cryptocurencies':'cryptocurrencies','koinex':'bitcoin',                'zebpay':'bitcoin','binance':'bitcoin','whtsapp':'whatsapp','reactjs':'framework','bittrex':'bitcoin','bitconnect':'bitcoin',                'bitfinex':'bitcoin','yourquote':'your quote','whyis':'why is','jiophone':'smartphone','dogecoin':'bitcoin','onecoin':'bitcoin',                'poloniex':'bitcoin','7700k':'cpu','angular2':'framework','segwit2x':'bitcoin','hashflare':'bitcoin','940mx':'gpu',                'openai':'framework','hashflare':'bitcoin','1050ti':'gpu','nearbuy':'near buy','freebitco':'bitcoin','antminer':'bitcoin',                'filecoin':'bitcoin','whatapp':'whatsapp', 'empowr':'empower','1080ti':'gpu','crytocurrency':'cryptocurrency','8700k':'cpu',                'whatsaap':'whatsapp','g4560':'cpu','payymoney':'pay money','fuckboys':'fuck boys','intenship':'internship','zcash':'bitcoin',                'demonatisation':'demonetization','narcicist':'narcissist','mastuburation':'masturbation','trignometric':'trigonometric',                'cryptocurreny':'cryptocurrency','howdid':'how did','crytocurrencies':'cryptocurrencies','phycopath':'psychopath',                'bytecoin':'bitcoin','possesiveness':'possessiveness','scollege':'college','humanties':'humanities','altacoin':'bitcoin',                'demonitised':'demonetized','brasília':'brazilia','accolite':'accolyte','econimics':'economics','varrier':'warrier','quroa':'quora',                'statergy':'strategy','langague':'language', 'splatoon':'game','7600k':'cpu','gate2018':'gate 2018','in2018':'in 2018',                'narcassist':'narcissist','jiocoin':'bitcoin','hnlu':'hulu','7300hq':'cpu','weatern':'western','interledger':'blockchain',                'deplation':'deflation', 'cryptocurrencies':'cryptocurrency', 'bitcoin':'blockchain cryptocurrency',
                "ain't": "is not", "aren't": "are not", "can't": "cannot", "'cause": "because",
                "could've": "could have", "couldn't": "could not", "didn't": "did not", "doesn't": "does not",
                "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                "he'd": "he would", "he'll": "he will", "he's": "he is", "how'd": "how did",
                "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would",
                "i'd've": "i would have", "i'll": "i will", "i'll've": "I will have", "i'm": "i am",
                "i've": "I have", "isn't": "is not", "it'd": "it would",
                "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is",
                "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
                "mightn't": "might not", "mightn't've": "might not have", "must've": "must have",
                "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not",
                "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
                "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                "so've": "so have", "so's": "so as", "this's": "this is", "that'd": "that would",
                "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                "there'd've": "there would have", "there's": "there is", "here's": "here is",
                "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                "they'll've": "they will have", "they're": "they are", "they've": "they have",
                "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have",
                "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
                "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have",
                "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not",
                "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
                "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have",
                "you'd": "you would", "you'd've": "you would have", "you'll": "you will",
                "you'll've": "you will have", "you're": "you are", "you've": "you have"}               


In [ ]:
import re

def clear_punct(text):
    for p in punctuation_list:
        if p in text:
            text = text.replace(p, " ")
    return text
print(clear_punct("Ha♌hEha.!fsd"))

def replace_word(text):
    for i in misspell_dict:
        if i in text:
            text = text.replace(i, misspell_dict[i])
    return text
print(replace_word("you're my favourite narcicist"))
                   
def clear_num(text):
    text = re.sub('[0-9]{5,}', '#####', text)
    text = re.sub('[0-9]{4}', '####', text)
    text = re.sub('[0-9]{3}', '###', text)
    text = re.sub('[0-9]{2}', '##', text)
    return text
print(clear_num("10 little indians"))

In [ ]:
# thực hiện các làm làm sạch sử liệu train và test
train_data['question_text'] = train_data['question_text'].apply(lambda x: x.lower())
train_data['question_text'] = train_data['question_text'].apply(lambda x: clear_num(x))
train_data['question_text'] = train_data['question_text'].apply(lambda x: replace_word(x))
train_data['question_text'] = train_data['question_text'].apply(lambda x: clear_punct(x))

test_data['question_text'] = test_data['question_text'].apply(lambda x: x.lower())
test_data['question_text'] = test_data['question_text'].apply(lambda x: clear_num(x))
test_data['question_text'] = test_data['question_text'].apply(lambda x: replace_word(x))
test_data['question_text'] = test_data['question_text'].apply(lambda x: clear_punct(x))

In [ ]:
# tách dữ liệu train thành 2 phần nhỏ hơn là tập train và tập validation kiểm thử
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(train_data['question_text'].values, train_data['target'].values, random_state = 1)
test_x = test_data['question_text'].values

# Xử lý dữ liệu
- Tokenize văn bản thành các từ
- Tạo bộ vocab sử dụng top 20000 token 
- Vector hóa các token
- Pad (hoặc truncate) để cho các câu đều có độ dài bằng nhau

In [ ]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

# mỗi mẫu có 100 từ
MAX_LENGTH = 100

# Tạo bộ vocab với dữ liệu train
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(train_x))

# Chuyển thành các sequence vector
train_x = tokenizer.texts_to_sequences(train_x)
val_x = tokenizer.texts_to_sequences(val_x)
test_x = tokenizer.texts_to_sequences(test_x)

# Pad độ dài các mẫu
train_x = sequence.pad_sequences(train_x, maxlen=MAX_LENGTH)
val_x = sequence.pad_sequences(val_x, maxlen=MAX_LENGTH)
test_x = sequence.pad_sequences(test_x, maxlen=MAX_LENGTH)

**Embeddings**

Sử dụng 3 bộ pretrained embeddings mà đề bài đã cho sẵn: glove, wiki-news, paragram

In [ ]:
from zipfile import ZipFile

zip_file = ZipFile(emb_path, 'r')
zip_file.extractall()
zip_file.namelist()

In [ ]:
glove_path = './glove.840B.300d/glove.840B.300d.txt'
wiki_path = './wiki-news-300d-1M/wiki-news-300d-1M.vec'
paragram_path =  './paragram_300_sl999/paragram_300_sl999.txt'

In [ ]:
num_features = 20000
emb_dim = 300

In [ ]:
# https://www.kaggle.com/gmhost/gru-capsule
def load_emb(word_index, emb_path, is_parag=False):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    # paragram uses utf8, different from others -> this function looks stupid
    if is_parag:
        emb_index = dict(get_coefs(*o.split(' ')) for o in open(emb_path, encoding='utf8', errors='ignore') if len(o)>100 and o.split(' ')[0] in word_index)
    else:
        emb_index = dict(get_coefs(*o.split(' ')) for o in open(emb_path) if len(o)>100 and o.split(' ')[0] in word_index )
    all_embs = np.stack(emb_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    emb_matrix = np.random.normal(emb_mean, emb_std, (num_features, embed_size))
    for word, i in word_index.items():
        if i >= num_features: continue
        emb_vector = emb_index.get(word)
        if emb_vector is not None: emb_matrix[i] = emb_vector
    return emb_matrix

In [ ]:
glove_emb = load_emb(tokenizer.word_index, glove_path)
wiki_emb = load_emb(tokenizer.word_index, wiki_path)
parag_emb = load_emb(tokenizer.word_index, paragram_path, True)
# Sử dụng bộ embedding là trung bình của cả 3 bộ trên
emb_matrix = np.mean([glove_emb, wiki_emb, parag_emb], axis = 0)

# Mô hình
Xây dựng theo mô hình RNN (https://www.tensorflow.org/text/tutorials/text_classification_rnn#create_the_model)

Về RNN: https://tricky-tax-444.notion.site/06-M-ng-n-ron-cho-chu-i-d-li-u-6e64974cae7a4bf098c98d196dc6534e#b599baf0f61744678f992007b0f01034

![](https://www.tensorflow.org/text/tutorials/images/bidirectional.png)

Xây dưng một mô hình đơn giản gồm các lớp:
- Embedding: sử dụng bộ embedding đã tính phía trên, freeze lớp này lại, vì em thấy nếu không thì sẽ khá tốn thời gian và tốn tài nguyên mà cũng không đem lại hiệu quả cao hơn mấy.
- Bidirectional: chỉ sử dụng như một lớp ở trong mô hình
- MaxPool
- Dense
- Dropout

In [ ]:
from tensorflow.python.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers import Bidirectional
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Conv1D
from tensorflow.python.keras.layers import GlobalMaxPool1D
from tensorflow.python.keras.layers import GRU


inputs = Input(shape=(MAX_LENGTH))
x = Embedding(input_dim=num_features, output_dim=emb_dim,weights=[emb_matrix], trainable=False)(inputs)
x = Bidirectional(GRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.1)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=inputs, outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]

model.summary()

In [ ]:
history = model.fit(train_x,train_y, epochs = 5, validation_data = (val_x,val_y), callbacks = callbacks, verbose = 1,batch_size = 256)
# Print results.
history = history.history
# print('Validation accuracy: {acc}, loss: {loss}'.format(acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

In [ ]:
pred = model.predict(test_x, batch_size = 256)
y_test = (pred[:,0] > 0.5).astype(np.int)
submission = pd.DataFrame({'qid': test_data['qid'], 'prediction': y_test})
submission.to_csv('submission.csv', index=False)